In [1]:
# Setup of libraries, mounting the Google Drive etc.

import os
import sys
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('gdrive/My Drive/Colab Notebooks')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Read the dataset from `.pkl` file


In [2]:
df = pd.read_pickle('cabspotting.pkl')
df.head()

latitude  longitude  occupied
user   time                                              
abboip 2008-05-17 14:12:10  37.75153 -122.39447         0
       2008-05-17 14:13:34  37.75149 -122.39447         0
       2008-05-17 14:14:34  37.75149 -122.39447         0
       2008-05-17 14:15:35  37.75149 -122.39446         0
       2008-05-17 14:41:43  37.75144 -122.39449         0

## Convert the dataset to a mapping of users to the strings of their movements

In [3]:
import datetime as dt

df.reset_index(inplace=True)

df['rebased_time'] = df['time'] - df['time'].min()
df['rebased_time'] = df['rebased_time'].dt.total_seconds()

df = df[['user', 'rebased_time', 'longitude', 'latitude']]

def extract_sequence(df):
    df.drop('user', axis=1, inplace=True)
    df.sort_values(by='rebased_time', inplace=True)
    return df.values

df = df.groupby('user').apply(extract_sequence)

df.head()

user
abboip      [[15126.000000000002, -122.39447, 37.751529999...
abcoij      [[47486.0, -122.41466000000001, 37.80346], [47...
abdremlu    [[11949.0, -122.39093000000001, 37.75521], [12...
abgibo      [[16.0, -122.4374, 37.7733], [61.0000000000000...
abjoolaw    [[13856.0, -122.39747, 37.75159], [13916.0, -1...
dtype: object

## Concatenate all user strings of data to one complete string of all

In [4]:
def to_single_sequence(series):
    data = None
    for _, item in series.iteritems():
        if data is None:
            data = item
        else:
            data = np.concatenate((data, item), axis=0)
    
    return data

data = to_single_sequence(df)
del df

print(data.shape)
print()
print(data)

(11219955, 3)

[[ 1.5126000e+04 -1.2239447e+02  3.7751530e+01]
 [ 1.5210000e+04 -1.2239447e+02  3.7751490e+01]
 [ 1.5270000e+04 -1.2239447e+02  3.7751490e+01]
 ...
 [ 2.0172080e+06 -1.2244239e+02  3.7756860e+01]
 [ 2.0172640e+06 -1.2244215e+02  3.7760470e+01]
 [ 2.0173250e+06 -1.2243687e+02  3.7760750e+01]]


## Standardisation
All three features in the data are scaled to have a `mean = 0` and a `standard deviation = 1`.

The Scaler is also saved to disk, so it can later be resued in a different Notebook.

In [5]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data)

data = scaler.transform(data)

print('Means: %f %f %f' % (scaler.mean_[0], scaler.mean_[1], scaler.mean_[2]))
print('Standard deviations: %f %f %f' % (scaler.var_[0], scaler.var_[1], scaler.var_[2]))
print()
print(data)

Means: 1017983.546543 -122.412432 37.763601
Standard deviations: 346406383099.499146 0.001280 0.002901

[[-1.70390864  0.50197561 -0.22410877]
 [-1.70376591  0.50197561 -0.22485143]
 [-1.70366397  0.50197561 -0.22485143]
 ...
 [ 1.69773582 -0.83724035 -0.12514886]
 [ 1.69783096 -0.83053309 -0.05812348]
 [ 1.69793461 -0.6829734  -0.05292484]]


In [62]:
with open('cabspotting_StandardScaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
    print('Scaler saved')

Scaler saved


## Convert the data to 32-bit floats
This is to correct an error I got earlier. Apparently the GRU layers can't handle 64-bit integers (doubles) as input.

In [6]:
print('Before: ', data.dtype)

Before:  float64


In [0]:
data = data.astype(np.float32)

In [8]:
print('After: ', data.dtype)

After:  float32


## Reshape and -order the data for input into the model (including X and y split)


In [9]:
data.shape

(11219955, 3)

In [11]:
BATCH_SIZE = 64
SEQ_LENGTH = 100

n_batches = (data.shape[0] - 1) // (BATCH_SIZE * SEQ_LENGTH)
rounded_length = n_batches * BATCH_SIZE * SEQ_LENGTH

X = data[:rounded_length].reshape([BATCH_SIZE, n_batches * SEQ_LENGTH, 3])
y = data[1:rounded_length + 1].reshape([BATCH_SIZE, n_batches * SEQ_LENGTH, 3])

print('X:', X.shape)
print('y:', y.shape)

X: (64, 175300, 3)
y: (64, 175300, 3)


In [12]:
def reshape_to_feed_order(data):
    tmp = data[:, :SEQ_LENGTH]
    for b in range(1, n_batches):
        tmp = np.concatenate([tmp, data[:, b * SEQ_LENGTH:(b + 1) * SEQ_LENGTH]])
    data = tmp
    del tmp
    
    return data

X = reshape_to_feed_order(X)
y = reshape_to_feed_order(y)

print('X:', X.shape)
print('y:', y.shape)

X: (112192, 100, 3)
y: (112192, 100, 3)


## Split into training and validations sets

In [13]:
n_train = int(n_batches * 0.8) * BATCH_SIZE
X_train = X[:n_train]
y_train = y[:n_train]
X_val = X[n_train:]
y_val = y[n_train:]

print('Train:', X_train.shape, y_train.shape)
print('Validation:', X_val.shape, y_val.shape)

Train: (89728, 100, 3) (89728, 100, 3)
Validation: (22464, 100, 3) (22464, 100, 3)


## Build the model

In [0]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(1024, return_sequences=True, stateful=True, input_shape=(None, 3), batch_size=batch_size),
        tf.keras.layers.TimeDistributed(tf.layers.Dense(3))
    ])
    return model

In [17]:
model = build_model(batch_size=BATCH_SIZE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (64, None, 1024)          3158016   
_________________________________________________________________
time_distributed (TimeDistri (64, None, 3)             3075      
Total params: 3,161,091
Trainable params: 3,161,091
Non-trainable params: 0
_________________________________________________________________


## Test the model's output (before training)

In [18]:
model.input

<tf.Tensor 'gru_input:0' shape=(64, ?, 3) dtype=float32>

In [19]:
# Test the shape of the output
test_batch = X[:BATCH_SIZE]
print('Input batch:', test_batch.shape)
test_predictions = model.predict(test_batch, batch_size=BATCH_SIZE)

print('Output batch:', test_predictions.shape)

model.reset_states()

Input batch: (64, 100, 3)
Output batch: (64, 100, 3)


In [0]:
test_batch = scaler.inverse_transform(test_batch)
test_predictions = scaler.inverse_transform(test_predictions)

In [23]:
print('Input: \n', test_batch[0, 0], test_batch[0, 1], test_batch[0, 2], '...')
print()
print('Next Predictions: \n', test_predictions[0, 0], test_predictions[0, 1], test_predictions[0, 2], '...')

Input: 
 [15125.984    -122.39447    37.75153] [15210.047    -122.39447    37.75149] [15270.047    -122.39447    37.75149] ...

Next Predictions: 
 [ 1.02200750e+06 -1.22414986e+02  3.77611847e+01] [ 1.0263729e+06 -1.2241620e+02  3.7759567e+01] [ 1.0294404e+06 -1.2241674e+02  3.7758549e+01] ...


## Training

In [0]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'onegru_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_new_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix)

In [26]:
hist = model.fit(X_train, y_train, epochs=30, batch_size=BATCH_SIZE, shuffle=False, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Train on 89728 samples, validate on 22464 samples
Epoch 1/30
89728/89728 [==============================] - 552s 6ms/step - loss: 0.0417 - val_loss: 0.0372
Epoch 2/30
89728/89728 [==============================] - 548s 6ms/step - loss: 0.0342 - val_loss: 0.0342
Epoch 3/30
89728/89728 [==============================] - 551s 6ms/step - loss: 0.0328 - val_loss: 0.0310
Epoch 4/30
89728/89728 [==============================] - 552s 6ms/step - loss: 0.0318 - val_loss: 0.0310
Epoch 5/30
89728/89728 [==============================] - 551s 6ms/step - loss: 0.0311 - val_loss: 0.0304
Epoch 6/30
89728/89728 [==============================] - 551s 6ms/step - loss: 0.2026 - val_loss: 0.2763
Epoch 7/30
89728/89728 [==============================] - 548s 6ms/step - loss: 0.1478 - val_loss: 0.1635
Epoch 8/30
89728/89728 [==============================] - 548s 6ms/step - loss: 0.1061 - val_loss: 0.1724
Epoch 9/30
89728/89728 [==============================] - 549s 6ms/step - loss: 0.1031 - val_loss: 0.1

## Save and plot the history
The history holds information on loss and validiation loss during training.

In [27]:
with open('onegru_history.pkl', 'wb') as file:
    pickle.dump(hist.history, file)
    print('History saved')

History saved


In [29]:
plt.plot(hist.history['loss'], label='Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

## Load and prepare the trained model for generating data

In [0]:
loaded_model = tf.keras.models.load_model('onegru_training_checkpoints/ckpt_new_29')

In [0]:
model = build_model(batch_size=1)

weights = loaded_model.get_weights()
model.set_weights(weights)

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (1, None, 1024)           3158016   
_________________________________________________________________
time_distributed_1 (TimeDist (1, None, 3)              3075      
Total params: 3,161,091
Trainable params: 3,161,091
Non-trainable params: 0
_________________________________________________________________


In [51]:
start = [324.0, -122.390085, 37.789910]    # Google Offices in San Francisco: 37.789910, -122.390085
generated = np.array([start]).astype(np.float32)
generated[-1:]
generated.dtype

dtype('float32')

## Generate geolocation data of one entity using the trained model
In the end the generated data is saved to disk in order to be reused from a different notebook.

In [0]:
N_GENERATE = 1000

start = [324.0, -122.390085, 37.789910]    # Google Offices in San Francisco: 37.789910, -122.390085
generated = np.array([start]).astype(np.float32)

model.reset_states()

for i in range(1, N_GENERATE):
    input = generated[-1:]
    input = scaler.transform(input)
    input = np.array([input])
    
    prediction = model.predict(input, batch_size=1)
    
    prediction = np.squeeze(prediction, axis=0)
    prediction = scaler.inverse_transform(prediction)
    
    generated = np.concatenate([generated, prediction])

In [60]:
generated.shape

(1000, 3)

In [61]:
generated

array([[ 3.2400000e+02, -1.2239008e+02,  3.7789909e+01],
       [-1.8735502e+06, -1.2234660e+02,  3.7805630e+01],
       [-4.7085100e+06, -1.2225939e+02,  3.7808739e+01],
       ...,
       [-7.2466465e+06, -1.2193810e+02,  3.7387192e+01],
       [-7.2466465e+06, -1.2193810e+02,  3.7387192e+01],
       [-7.2466465e+06, -1.2193810e+02,  3.7387192e+01]], dtype=float32)

In [63]:
with open('onegru_generated_test.pkl', 'wb') as file:
    pickle.dump(generated, file)
    print('Generated data saved')

Generated data saved
